In [1]:
import re
script_words = {}

def read_text(movie):
    textline = []
    textname = movie+'.txt'
    lines = open(textname,'r').readlines()
    for line in lines:
        textline.append(line.strip())
    return textline

#'the-godfather'
#'the-fault-in-our-stars'
file_names = ['the-godfather','the-hangover','the-lord-of-the-rings','star-wars','ghostbusters']
for movie in file_names:
    script_words[movie] = read_text(movie)


In [2]:
import pandas as pd
import spacy
from collections import Counter
nlp = spacy.load('en')
def named_entity_counts(document,named_entity_label):   
    ## Function that outputs a Counter object of human entities found\n",
    occurrences = [ent.string.strip() for ent in document.ents if ent.label_ == named_entity_label and ent.string.strip()]
    return Counter(occurrences)

def parse_text(movie):
    name_text = movie + '.txt'
    text2 = open(name_text).read()
    doc = nlp(text2)
    parsed_script = doc  
    entity_type = 'PERSON' 
    number_of_entities = 5
    Entities=pd.DataFrame(named_entity_counts(parsed_script,entity_type).most_common(number_of_entities),columns=["Character","Count"])
    Entities['Movie'] = movie
    return Entities

result = pd.DataFrame()
for file in file_names:
    df = parse_text(file)
    result = result.append(df,ignore_index = True)

In [3]:
def dict_names (Character):
    name_dict = {}
    for character in Character:
        character = character.upper()
        if (character in name_dict.keys()):
            continue
        if len(character.split()) >=2:
            full_name = character.split()
            name_dict[full_name[0]] = character
        else:
            name_dict[character] = ''
    return name_dict

dictionary_names = dict_names (result['Character'])

In [4]:
name_movie = result.set_index('Character').to_dict()['Movie']
name_movie2 = {}
for name in dictionary_names.keys():
    for n in name_movie.keys():
        if name in n.upper():
            name_movie2[name] = name_movie[n]   
print (name_movie2)


{'MICHAEL': 'the-godfather', 'KAY': 'the-godfather', 'SONNY': 'the-godfather', 'MIKE': 'the-godfather', 'VICK': 'the-hangover', 'ALAN': 'the-hangover', 'DOUG': 'the-hangover', 'GANDALF': 'the-lord-of-the-rings', 'SAM': 'the-lord-of-the-rings', 'BOROMIR': 'the-lord-of-the-rings', 'ARAGORN': 'the-lord-of-the-rings', 'FRODO': 'the-lord-of-the-rings', 'LUKE': 'star-wars', 'CHEWIE': 'star-wars', 'LEIA': 'star-wars', 'VADER': 'star-wars', 'LANDO': 'star-wars', 'VENKMAN': 'ghostbusters', 'SPENGLER': 'ghostbusters', 'STANTZ': 'ghostbusters', 'DANA': 'ghostbusters'}


In [5]:
def people_says (textlist,character):
    sublist = []
    words_list = []
    for i in range(len(textlist)):
        if (textlist[i].strip() == character):
            j = i + 1
            while (textlist[j]!= ''):
                j += 1
            sublist.append(textlist[i+1:j])
    words_list.append(sublist)
#     for character in dictionary_names.values():
#         if character != '':
#             for i in range(len(textlist)):
#                 if (textlist[i].strip() == character):
#                     j = i + 1
#                     while (textlist[j]!= ''):
#                         j += 1
#                     sublist.append(textlist[i:j])
#         words_list.append(sublist)
    return words_list
def char_names (movie):
    n = []
    for name in name_movie2:
        if name_movie2[name] == movie:
            n.append(name)
    return n

for movie in file_names:
    for name in char_names(movie):
        file_name = name+'.txt'
        file = open(file_name,"w")
        file.write(str(people_says(script_words[movie],name)))
file.close()


In [6]:
import json
import requests

headers = {
    'Content-Type': 'text/plain;charset=utf-8',
    'Accept': 'application/json',
}

params = (
    ('version', '2017-10-13'),
)

def json_parser(text_name):
    data = open(text_name, 'rb').read()
    response = requests.post('https://gateway.watsonplatform.net/personality-insights/api/v3/profile?version=2017-10-13', headers=headers, data=data, auth=('apikey', 'QoK4TRsFxHfWUW3d-_KV4DOujgALshtnrzPolJ6BY_Wk'))
    parsed = json.loads(response.text)

    trait_dict = {}

    for i in range(len(parsed)-2):
        trait_dict[(parsed['personality'][i]['name'])] = parsed['personality'][i]['percentile']

        for j in (range(len(parsed['personality'][i]['children']))):
            trait_dict[(parsed['personality'][i]['children'][j]['name'])] = parsed['personality'][i]['children'][j]['percentile']
    return trait_dict


In [8]:
personality = []
#if re-run the function again, need to un-comment these below
dictionary_names.pop('MIKE')
dictionary_names.pop('CHEWIE')
for name in dictionary_names.keys():
    txtname = name + '.txt'
    personality.append(json_parser(txtname))


In [15]:
from cloudant import cloudant_iam,query

# Authenticate using an IAM API key
ACCOUNT_NAME = '9f18399f-0e0a-4464-9cae-afcc09328dbe-bluemix'
API_KEY = 'j7H-txptJqOJ9G1PQ01hIHZYcs2oieznEkt6qWvGQBio'
DB_NAME = 'elmo_characters'

def put_character(db, **kargs):
    db.create_document(kargs)

def get_character(db, c_id):
    return db[c_id]

def get_character_id(movie, character):
   return ":".join([movie.lower(), character.lower()])

def search(db, movie, lmt = 10):
    qr = query.Query(db, selector={"movie": movie})
    return list(qr(limit = lmt)['docs'])

def connect_db():
    client = cloudant_iam(ACCOUNT_NAME, API_KEY, connect=True)
    db_session = client.session()
    if not DB_NAME in client.all_dbs():
        db = client.create_database(DB_NAME)

    if not db.exists():
        return None

    return client[DB_NAME]

def search_movie(movie):
    with cloudant_iam(ACCOUNT_NAME, API_KEY, connect=True) as client:
        db = None
        if not DB_NAME in client.all_dbs():
            db = client.create_database(DB_NAME)
        else:
            db = client[DB_NAME]
            
        movie = movie.strip()
        return search(db, movie.replace(' ', '-')) 

if __name__ == "__main__":

    with cloudant_iam(ACCOUNT_NAME, API_KEY, connect=True) as client:
        db = None
        dblist = client.all_dbs()
    
        if not DB_NAME in dblist:
            db = client.create_database(DB_NAME)
        else:
            db = client[DB_NAME] 
        
        for name in dictionary_names.keys():
            
            txtname = name + '.txt'
            c_id = get_character_id(name_movie2[name], name)
            put_character(db, _id=c_id, movie = name_movie2[name],character=name, personality=json_parser(txtname))
        
        for doc in search_movie('the lord of the rings'):
            print (doc['personality'])




{'Openness': 0.9119848877026207, 'Adventurousness': 0.6038497217061395, 'Artistic interests': 0.9578941990319867, 'Emotionality': 0.7027423921357344, 'Imagination': 0.9634975651268665, 'Intellect': 0.9822857010520005, 'Authority-challenging': 0.33348480428548877, 'Conscientiousness': 0.41462612078798344, 'Achievement striving': 0.6811621180787211, 'Cautiousness': 0.5230057936924101, 'Dutifulness': 0.39375873560838437, 'Orderliness': 0.5042439682599319, 'Self-discipline': 0.413560533230124, 'Self-efficacy': 0.5858992544355176, 'Extraversion': 0.07263553593037803, 'Activity level': 0.505002270118807, 'Assertiveness': 0.4857583572660254, 'Cheerfulness': 0.362725520684789, 'Excitement-seeking': 0.6093242713415268, 'Outgoing': 0.08471101750022664, 'Gregariousness': 0.012041229932974795, 'Agreeableness': 0.09243758234351301, 'Altruism': 0.7112220356523786, 'Cooperation': 0.22203802713402698, 'Modesty': 0.9206981101165445, 'Uncompromising': 0.8071769103671835, 'Sympathy': 0.98054449254638, 'T

In [31]:
from __future__ import print_function
from __future__ import print_function
import json, uuid
from ibm_watson import AssistantV2
NUMBER_OF_TRAITS = 28

def least_squares_matcher(chatbot_percentiles, list_of_percentiles,movie_characters):
    least_squares = []
    i = 0
    j = 0

    print (len(list_of_percentiles))
    while j < len(list_of_percentiles):
        cur_least_square = 0
        
        while i < 10: 
            
            diff = chatbot_percentiles[i] - list_of_percentiles[j][i]
            square = diff * diff
            cur_least_square += square
            i += 1

        least_squares.append(cur_least_square)
        j += 1
    

    least_square_index = (least_squares.index(min(least_squares)))

    return (movie_characters[least_square_index])
        

def load_credential(authfile = None):
    if authfile == None:
        authfile = 'ibm-credentials.env'

    with open(authfile) as f:
        cred = {}
        for line in f:
            parts = line.strip('\r').strip('\n').split("=")

            if len(parts) == 2:
                cred[parts[0]] = parts[1]
        return cred;

    return None

def create_elmo(credential):
   # If service instance provides API key authentication
   return AssistantV2(
              version='2018-09-20',
              # url is optional, and defaults to the URL below. Use the correct URL for your region.
              url='https://gateway.watsonplatform.net/assistant/api',
              iam_apikey=credential['ASSISTANT_IAM_APIKEY'])

#########################
# Sessions
#########################
def create_session(assistant, ass_id):
    session = assistant.create_session(ass_id).get_result()
    print(json.dumps(session, indent=2))
    return session

#########################
# Message
#########################
def send_message(assistant, ass_id, sess_id, msg):
    message = assistant.message(
    ass_id,
    sess_id,
    input={'text': msg}, #'What\'s the weather like?'},
    context={'metadata': {'deployment': 'myDeployment'
           }
       }).get_result()
   #print(json.dumps(message, indent=2))
    return message['output']['generic'][0]['text']

def delete_session(assistant, ass_id, sess_id):
    return assistant.delete_session(ass_id, sess_id).get_result()

if __name__ == '__main__':
    credential = load_credential()
    
    if not credential:
        pass

    elmo = create_elmo(credential)
    ass_id = "70f87349-4762-41fb-a263-5dfc9626ad53"
    sess_id = create_session(elmo, ass_id)["session_id"]

    history = []
    dialog = "hello"
    msg = ""
    movie = "the-godfather"
    movie_characters = []

   #CHATBOT STRUCTURES
    chatbot_dict = {} #dictionary of traits->percentiles for user conversation with chatbot
    chatbot_percentiles = []  #list of percentiles for user conversation with chatbot

   #MOVIE CHARACTER STRUCTURES
    list_of_dicts = []  #list of dictionaries of traits->percentiles (dictionary is for each character)
    list_of_percentiles = []  #list of lists of percentiles (list is for each character)
    smaller_list = [] #for parsing purposes
    movie_characters = []


    while not "bye" in msg.lower() and not "see you" in msg.lower():
        msg = send_message(elmo, ass_id, sess_id, dialog)


        if "Bingo! this is it" in msg:
            movie = history[-1].split("movie")[1].strip()
            movie_characters = char_names(movie)
            history.append(msg)
            
        
        elif "Character" in msg or "Match character" in msg or "Search character" in msg:
            if len(movie_characters)  == 0:
                movie_characters = char_names(movie)

            import json
            import requests

            headers = {'Content-Type': 'text/plain;charset=utf-8','Accept': 'application/json',}

            params = (('version', '2017-10-13'),)

            data = open('history.txt', 'rb').read()
            response = requests.post('https://gateway.watsonplatform.net/personality-insights/api/v3/profile?version=2017-10-13', headers=headers, data=data, auth=('apikey', 'QoK4TRsFxHfWUW3d-_KV4DOujgALshtnrzPolJ6BY_Wk'))
            parsed = json.loads(response.text)

           #parses through chatbot dictionary traits->percentiles to generate list of percentiles 
            for i in range(len(parsed)-2):
                chatbot_dict[(parsed['personality'][i]['name'])] = parsed['personality'][i]['percentile']
 
            for j in (range(len(parsed['personality'][i]['children']))):
               chatbot_dict[(parsed['personality'][i]['children'][j]['name'])] = parsed['personality'][i]['children'][j]['percentile']

            for trait in chatbot_dict:
                chatbot_percentiles.append(chatbot_dict[trait])
            
            for doc in search_movie(movie):
                list_of_dicts.append(doc['personality'])

            for item in list_of_dicts:
                for trait in item:
                    smaller_list.append(item[trait])  
                list_of_percentiles.append(smaller_list)
                smaller_list=[]
            
            print ('You are most like ')
            msg = least_squares_matcher(chatbot_percentiles, list_of_percentiles,movie_characters)



           # do the personal insights/api/v3/profile# find the best character
        dialog = input("Elmo: %s\nMe: " %(msg.strip('\r').strip('\n')))
        history.append(dialog)

        file_name = 'history'+'.txt'
        file = open(file_name,"w")
        file.write(str(history))
        file.close()

    delete_session(elmo, ass_id, sess_id)

{
  "session_id": "65ffa3a6-dcfa-486b-bbfb-2e32f77ec14a"
}
Elmo: What would be the most amazing adventure to go on?
Me: switch movie the lord of the rings
Elmo: Bingo! this is it.
Me: coding
Elmo: What would be your first question after waking up from being cryogenically frozen for 100 years?
Me: history.append(msg)
Elmo: What’s your favorite genre of book or movie?
Me: coding
Elmo: What do you wish you knew more about?
Me: coding
Elmo: What would be your ideal way to spend the weekend?
Me: coding
Elmo: What’s the best single day on the calendar?
Me: coding
Elmo: I didn't understand. You can try rephrasing.
Me: coding
Elmo: What shows are you into?
Me: coding
Elmo: If you had unlimited funds to build a house that you would live in for the rest of your life, what would the finished house be like?
Me: coding
Elmo: What takes up too much of your time?
Me: coding
Elmo: What state or country do you never want to go back to?
Me: coding
Elmo: Can you reword your statement? I'm not understandi

KeyError: 'personality'

{
  "session_id": "510e9965-5ef5-42db-bb5b-1173fe1d2532"
}


NameError: name 'send_message' is not defined